<a href="https://colab.research.google.com/github/wjdgoruds2/AI-_assignment/blob/master/HTN_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import copy,sys

class State():
  def __init__(self,name):
    self.__name__=name
class Goal():
  def __init__(self,name):
    self.__name__=name

def print_state(state,indent=4):
  if state != False:
    for(name,val) in vars(state).items():
      if name !='__name__':
        for x in range(indent):
          sys.stdout.write(' ')
        sys.stdout.write(state.__name__+'.'+name)
        print(' =',val)
  else:print('False')

def print_goal(goal,indent=4):
  if goal !=False:
    for(name,val) in vars(goal).items():
      if name !='__name__':
        for x in range(indent):
          sts.stdout.write(' ')
        sys.stdout.write(goal.__name__+'.'+name)
        print(' =',val)
  else:print('False')

def forall(seq,cond):
  for x in seq:
    if not cond(x):
      return False
  return True

def find_if(cond,seq):
  for x in seq:
    if cond(x):
      return x
    return None

operators={}
methods={}

def declare_operators(*op_list):
  operators.update({op.__name__:op for op in op_list})
  return operators

def declare_methods(task_name,*method_list):
  methods.update({task_name:list(method_list)})
  return methods[task_name]

def print_operators(olist=operators):
  print('연산자:',', '.join(olist))

def print_methods(mlist=methods):
  print('{:<14}{}'.forma('태스크:','메소드:'))
  for task in mlist:
    print('{:<14}'.format(task)+', '.join([f.__name__ for f in mlist[task]]))

def pyhop(state,tasks,verbose=0):
  print('태스크: ',tasks)
  if verbose>0:
    print('verbose={}:\n상태= {}\n태스크={}'.format(verbose,state.__name__,tasks))
  result=seek_plan(state, tasks, [],0, verbose)
  if verbose>0:
    print('결과: ',result,'\n')
  return result

def seek_plan(state, tasks, plan, depth, verbose=0):
  if verbose>1:
    print('깊이 {} 태스크 {}'.format(depth,tasks))
  if tasks==[]:
    if verbose>2:
      print('깊이 {} 반복된 계획 {}'.format(depth,plan))
    return plan
  task1=tasks[0]
  if task1[0] in operators:
    if verbose>2:
      print('깊이 {} 행동 {}'.format(depth,task1))
    operator=operators[task1[0]]
    newstate=operator(copy.deepcopy(state),*task1[1:])
    if verbose>2:
      print('깊이 {} 새로운 상태:'.format(depth))
      print_state(newstate)
    if newstate:
      solution=seek_plan(newstate, tasks[1:],plan+[task1],depth+1,verbose)
      if solution !=False:
        return solution
  if task1[0] in methods:
    if verbose>2:
      print('깊이 {} 메소드 {}'.format(depth,task1))
    relevant=methods[task1[0]]
    for method in relevant:
      subtasks=method(state,*task1[1:])
      if verbose>2:
        print('깊이 {} 새로운 태스크: {}'.format(depth,subtasks))
      if subtasks !=False:
        solution=seek_plan(state, subtasks+tasks[1:],plan,depth+1,verbose)
        if solution !=False:
            return solution
  if verbose>2: print ('깊이 {} 실패'.format(depth))
  return False

def travel_by_foot(state,a,x,y):
  if state.dist[x][y] <=4:
    return [('walk',a,x,y)]
  return False

def travel_by_taxi(state,a,x,y):
   if state.cash[a]>=1.5+0.5*state.dist[x][y]:
    return [('call_taxi',a,x),('ride_taxi',a,x,y),('pay_driver',a)]
   return False


declare_methods('travel',travel_by_foot,travel_by_taxi)

def walk(state,a,x,y):
  if state.loc[a] == x:
    state.loc[a]==y
    return state
  else:
    return False

def call_taxi(state,a,x):
  state.loc['taxi']=x
  return state

def ride_taxi(state,a,x,y):
  if state.loc['taxi']==x and state.loc[a]==x:
    state.loc['taxi']=y
    state.loc[a]=y
    state.owe[a]=1.5+0.5*state.dist[x][y]
    return state
  else:
    return False

def pay_driver(state,a):
  if state.cash[a]>=state.owe[a]:
    state.cash[a] = state.cash[a]-state.owe[a]
    state.owe[a]=0
    return state
  else:
    return False

declare_operators(walk,call_taxi,ride_taxi,pay_driver)

state1=State('state1')
state1.loc={'me':'home'}
state1.cash={'me':20}
state1.owe={'me':0}
state1.dist={'home':{'park':8},'park':{'home':8}}

pyhop(state1,[('travel','me','home','park')],verbose=2)

태스크:  [('travel', 'me', 'home', 'park')]
verbose=2:
상태= state1
태스크=[('travel', 'me', 'home', 'park')]
깊이 0 태스크 [('travel', 'me', 'home', 'park')]
깊이 1 태스크 [('call_taxi', 'me', 'home'), ('ride_taxi', 'me', 'home', 'park'), ('pay_driver', 'me')]
깊이 2 태스크 [('ride_taxi', 'me', 'home', 'park'), ('pay_driver', 'me')]
깊이 3 태스크 [('pay_driver', 'me')]
깊이 4 태스크 []
결과:  [('call_taxi', 'me', 'home'), ('ride_taxi', 'me', 'home', 'park'), ('pay_driver', 'me')] 



[('call_taxi', 'me', 'home'),
 ('ride_taxi', 'me', 'home', 'park'),
 ('pay_driver', 'me')]